# Food project

## Discovery (identificazione del problema e degli obiettivi)

Negli ultimi secoli la popolazione mondiale è aumentata notevolmente e secondo l'ONU (fonte del 2019), nel 2050 potrebbe arrivare a quota *10 miliardi*. 

In maniera direttamente proporzionale **sono aumentate anche le domande di cibo, energia ed acqua**, per soddisfare questo bisogno. 

Sfortunamente, i costanti monitoraggi sul cambiamento climatico suggeriscono che il clima della Terra sta cambiando pericolosamente e *parte di questo effetto è direttamente correlato alla produzione agricola e all'allevamento*:

- la produzione alimentare richiede acqua ed energia; 
- la produzione energetica tradizionale richiede risorse idriche; 
- l'agricoltura fornisce una potenziale fonte di energia e tanta acqua. 
 
Garantire che nel mondo tutte le persone abbiano accesso a una dieta nutriente *in modo sostenibile* è una delle maggiori sfide che dobbiamo affrontare. 


> **L'obiettivo di questo notebook** è quello di sfruttare alcuni dati inerenti alla *produzione e distribuzione mondiale di alimenti e mangimi*, per valutarne il loro impatto sull'ambiente in termini di *utilizzo di risorse idriche ed emissioni di gas serra*.

### Principali spunti e domande che guideranno questa analisi:

1. Il confronto tra produzione di cibo e mangime a livello mondiale

2. Quali paesi sono i maggiori produttori di cibo e mangimi?

3. Quali paesi sono cresciuti maggiormente in termini di produzione di cibo e/o magimi? 

4. Sono presenti interessanti outlier nei dati per quanto riguarda la produzione?

5. Quali tipi di alimenti hanno un impatto maggiormente negativo sull'ambiente?

    - Confronto dell'impatto ecologico tra: alimenti a base animale, a base vegetale e ricchi di proteine.
>
6. Quale fase della produzione alimentare contribuisce maggiormente all'emissione di gas serra?

7. Quali tipi di produzione alimentare dovrebbero essere incoraggiati per il consumo di una dieta maggiormente sostenibile per l'ambiente?

---
---

## Data selection

Questo progetto sfrutta due datasets caricati su Kaggle: 

- Il [primo](https://www.kaggle.com/datasets/dorbicycle/world-foodfeed-production), fornito dalla FAO (Food and Agriculture Organization of the United Nations), mostra una panoramica della produzione alimentare mondiale dal 1961 al 2013, concentrandosi su un confronto tra alimenti prodotti per il consumo umano e mangimi prodotti per l'allevamento di animali.

- Il [secondo](https://www.kaggle.com/datasets/selfvivek/environment-impact-of-food-production) contiene i valori di utilizzo dell'acqua e le emissioni di gas serra necessari per la realizzazione dei 43 alimenti più comuni prodotti nel mondo.

### Import di dati, moduli e funzioni utili

In [2]:
# Librerie fondamentali
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Librerie utili
from zipfile import ZipFile
from fuzzywuzzy import process
import fuzzywuzzy

pd.set_option('display.max_columns', None)

c:\Users\emanu\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Funzione personalizzata

def zip_extractor():
    """This function looks for zip files and extracts them in the same file path of this script, or in a folder provided 
    with an input. If the folder doesn't exist, it will be created with the provided (capitalized) name."""
    
    destination = input('Where do you want to put the extracted files?\n\
Press enter if you want to extract files in the current path, or write a folder name: ').capitalize()

    if destination == '':
        pass
    elif destination not in os.listdir():
        os.mkdir(destination)

    for dirname, _, filenames in os.walk(os.getcwd()):
        if dirname != 'Zip files':
            for filename in filenames:
                if '.zip' in filename:
                    with ZipFile(filename, "r") as zip:
                        zip.extractall(f'{destination}')

    print("Extraction: Done")

In [4]:
# Importo i files zip dei due dataset usando le API di Kaggle e li estraggo in una cartella

# !kaggle datasets download -d dorbicycle/world-foodfeed-production
# !kaggle datasets download -d selfvivek/environment-impact-of-food-production
# zip_extractor()

In [5]:
# Dataset della produzione di cibi/mangimi dei vari paesi del mondo
fao_df = pd.read_csv('Data/Fao.csv')
df_prod = fao_df.copy()

# Dataset delle emissioni di gas serra e dei consumi di acqua per la produzione di alimenti
emission_df = pd.read_csv('Data/Food_Production.csv')
df_emiss = emission_df.copy()

---
---
## Data cleaning

### Dataset della produzione di cibi/mangimi dei vari paesi del mondo

In [6]:
df_prod.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


#### Valori nulli

Procedo verificando la presenza di valori nulli (*NaN*) nel dataset, o altri valori incoerenti. Se effettivaemente presenti, approfondisco l'analisi per sapere quanti sono e in quali colonne sono maggiormente frequenti. Inoltre, proverò a fare un *imputation*, ovvero a stabilire il motivo della loro presenza. 

In [7]:
print(f'Dimensioni del dataframe: {df_prod.shape}')

# Quante celle contengono valori nulli rispetto alle celle totali?
total_cells = np.product(df_prod.shape)
print(f'Numero totale di celle: {total_cells}')

total_missing = df_prod.isnull().sum().sum()
print(f'Numero totale di celle con valori nulli: {total_missing}')

percent_missing = round(total_missing/total_cells * 100, 2)
print(f'\nPercentuale di valori nulli in questo Dataframe: {percent_missing}%')

Dimensioni del dataframe: (21477, 63)
Numero totale di celle: 1353051
Numero totale di celle con valori nulli: 117450

Percentuale di valori nulli in questo Dataframe: 8.68%


In [8]:
pd.set_option('display.max_rows', 63)

# Costruisco una tabella riassuntiva sui Dtype, valori unici e valori nulli di tutte le colonne
print("Dtype, Valori unici e Valori mancanti(%) di ciascuna colonna:")

df_prod_info= pd.DataFrame({"Dtype": df_prod.dtypes, 
                            "Valori unici": df_prod.nunique(),
                            "Valori mancanti(%)": round(df_prod.isnull().sum()/df_prod.shape[0]*100, 2)
                            }).rename_axis('Colonne', axis='rows')                       

df_prod_info

Dtype, Valori unici e Valori mancanti(%) di ciascuna colonna:


,Dtype,Valori unici,Valori mancanti(%)
Colonne,,,
Area Abbreviation,object,169,0.00
Area Code,int64,174,0.00
Area,object,174,0.00
Item Code,int64,117,0.00
Item,object,115,0.00
Element Code,int64,2,0.00
Element,object,2,0.00
Unit,object,1,0.00
latitude,float64,173,0.00


Si vede chiaramente che la maggior parte dei valori nulli presenti nel dataset è concentrata nelle colonne *tra il 1961 e il 1991*. Come si può leggere tra la documentazione nella pagina Kaggle dove è fornito il dataset:

>The Food Balance sheet's data was relatively complete. A few countries that do not exist anymore, such as Czechoslovakia, were deleted from the database. Countries which were formed lately such as South Sudan were kept, even though they do not have all full data going back to 1961. [...]

Quindi, in un certo senso, sembrerebbe che particolari *evoluzioni geopolitiche di alcuni paesi* siano la causa dell'indisponibilità dei dati.  

Prendo un campione a caso dove sono presenti questi valori nulli, cercando di fare chiarezza e di confermare le informazioni contenute nella documentazione del dataset.  

In [9]:
# Creo una lista con gli indici delle righe che contengono i valori nulli
rows_with_missing = df_prod.loc[df_prod.isnull().any(axis=1)].index.to_list()
print(f'\nNumero di righe con valori nulli: {len(rows_with_missing)}')

# Imposto un random seed per rendere pseudorandomica la riproducibilità di questa cella
np.random.seed(0)

# Utilizzo la lista per creare un dataframe avente solo le righe che contengono valori nulli
df_null_rows = df_prod.loc[rows_with_missing]

# Seleziono un campione di 5 elementi
df_null_rows.sample(5)


Numero di righe con valori nulli: 3539


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
17328,SVK,199,Slovakia,2517,Millet and products,5521,Feed,1000 tonnes,48.67,19.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1,1
13020,MNE,273,Montenegro,2617,Apples and products,5142,Food,1000 tonnes,42.71,19.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,19.0,34.0,38.0,33.0,27.0,14,13
17526,SVN,198,Slovenia,2733,Pigmeat,5142,Food,1000 tonnes,46.15,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,81.0,84.0,75.0,70.0,73.0,77.0,87.0,76.0,82.0,78.0,88.0,88.0,88.0,89.0,84.0,84.0,80.0,81.0,76.0,69,58
11128,LVA,119,Latvia,2656,Beer,5142,Food,1000 tonnes,56.88,24.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,53.0,66.0,67.0,66.0,73.0,82.0,100.0,99.0,105.0,134.0,142.0,124.0,144.0,147.0,158.0,156.0,146.0,164.0,172.0,163,156
16103,RUS,185,Russian Federation,2560,Coconuts - Incl Copra,5142,Food,1000 tonnes,61.52,105.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,2.0,2.0,10.0,10.0,21.0,28.0,42.0,45.0,47.0,52.0,45.0,59.0,42.0,39.0,48.0,63.0,57,63


Possiamo vedere che in questi 5 paesi selezionati a caso ('Slovacchia', 'Montenegro', 'Slovenia', 'Lettonia' e 'Federazione Russa'), in effetti i dati di produzione sono mancanti dal 1961 al 1991 (il Montenegro fino al 2005). 

Cercando informazioni sulla storia geopolitica di questi paesi (fonte: [Wikipedia](https://it.wikipedia.org/wiki/Pagina_principale)) si può leggere:

- *Slovacchia*: [...] La Repubblica Slovacca e la Repubblica Ceca sono nate il 1º gennaio 1993 dalla divisione, sancita dal parlamento della Cecoslovacchia, che **già dal 1990 aveva assunto il nome di Repubblica Federale Ceca e Slovacca**;

- *Montenegro*: [...] Fino al 2 giugno 2006 la Repubblica del Montenegro è stata unita alla Repubblica di Serbia con il nome di Serbia e Montenegro. **Dal 3 giugno 2006 il Montenegro è diventato uno Stato indipendente**;

- *Slovenia*: [...] Dal 1º maggio 2004 la Slovenia è membro dell'Unione europea e la valuta nazionale, dal 1º gennaio 2007, è l'euro, che ha sostituito il tallero sloveno, adottato **nel 1991 dopo l'indipendenza** [...];

- *Lettonia*: [...] L'URSS riconobbe **la Lettonia come Stato indipendente il 6 settembre 1991** [...];

- *Federazione Russa*: [...] Nell'agosto del 1991 il tentato colpo di Stato militare per deporre Gorbačëv e preservare l'Unione Sovietica portò invece alla fine del Partito Comunista dell'Unione Sovietica. Nonostante la contraria volontà espressa dal popolo, **il 26 dicembre 1991 l'Unione Sovietica si dissolse in quindici Stati post-sovietici**.

Questo risultato è in linea con la descrizione riportata dalla documentazione del dataset, giustificando il retroscena che riguarda la presenza di valori mancanti.

**È conveniente eliminare dal dataframe le righe che contengono valori nulli**, oppure si rischia di **perdere un numero eccessivo di informazioni?**

In [10]:
# Creo un dataframe droppando le righe che contengono valori nulli 
# 'rd' sta per 'row dropped'
df_prod_rd = df_prod.dropna()

# Calcolo la percentuale di dati eliminati rispetto dal dataframe originale
rows_removal_perc = round((1-df_prod_rd.shape[0]/df_prod.shape[0])*100,2)

# Stampo le informazioni sul numero di righe totali e la shape dei due dataframes
print(f"\nRighe del dataframe originale: {df_prod.shape[0]}. Shape del dataframe originale: {df_prod.shape}")
print(f"Righe del dataframe dopo aver rimosso i valori mancanti: {df_prod_rd.shape[0]}.\
Shape del dataframe modificato: {df_prod_rd.shape}")
print(f'\nLa rimozione delle righe che contengono valori NaN ha eliminato il {rows_removal_perc}% dei dati!')


Righe del dataframe originale: 21477. Shape del dataframe originale: (21477, 63)
Righe del dataframe dopo aver rimosso i valori mancanti: 17938.Shape del dataframe modificato: (17938, 63)

La rimozione delle righe che contengono valori NaN ha eliminato il 16.48% dei dati!


Avrei anche potuto sostituire i valori NaN con 0 tramite *fillna()*. Tuttavia, voglio visualizzare la produttività dei vari paesi nell'arco temporale più lungo possibile. Inoltre, secondo me, operando in questo modo **si perdono quantità tollerabili di dati**. Quindi, continuerò lo studio di questo dataset adottando questa procedura.

#### Valutazione di dati incoerenti 

##### Sistemo la colonna 'Area'

In [11]:
# Uso una nomeclatura più semplice per richiamare il dataframe
df = df_prod_rd.copy()

In [12]:
# Creo una serie per vedere se alcune etichette di 'Area Abbreviation' sono state utilizzate per più di un paese
find_duplicates = df.groupby('Area Abbreviation')['Area'].nunique()
find_duplicates[find_duplicates > 1]

Area Abbreviation
CHN    4
Name: Area, dtype: int64

In [13]:
# Uso l'etichetta 'CHN' per creare un filtro
chn_filter = df['Area Abbreviation'].isin(['CHN'])

# Uso il filtro per estrapolare un dataframe composto solo da paesi con etichetta 'CHN'
chn_df = df.loc[chn_filter]

# Creo una lista dall'array dei valori unici nella colonna 'Area' 
chn_countries = chn_df.Area.unique().tolist()

# Print informativo
print(f"Lista dei Paesi che condividono l'etichetta 'CHN':\n{chn_countries}\n\n\
Sostituirò questi nomi con:\n'China'")

# Sostituisco i nomi delle località presenti nella lista chn_countries con 'China' usando un dictionary comprehension
df.Area.replace({n:'China' for n in chn_countries}, inplace=True)

Lista dei Paesi che condividono l'etichetta 'CHN':
['China, Hong Kong SAR', 'China, Macao SAR', 'China, mainland', 'China, Taiwan Province of']

Sostituirò questi nomi con:
'China'


Uno strumento utilissimo per controllare contemporaneamente tutte le incongruenze tra gli elementi della colonna 'Area' è *fuzzywuzzy*. 

Questo strumento utilizza la [Distanza di Levenshtein](https://it.wikipedia.org/wiki/Distanza_di_Levenshtein), per misurare le differenze fra due stringhe e determinare quindi quanto due stringhe siano simili tra loro. 

La mia strategia è la seguente:

- Creo una lista dei valori unici dei paesi racchiusi nella colonna 'Area';

- Imposto un ciclo che itera per ogni paese della lista, restitundo a sua volta una lista di tuple composte da due elementi: nome del paese di confronto e punteggio;

    - La prima tupla ha punteggio massimo (100) poiché è il confronto dell'elemento che sta attualmente iterando nel ciclo con sè stesso;
> 
- Dalla seconda tupla in poi, invece, si ritrovano le effettive somiglianze tra l'elemento che sta attualmente iterando nel ciclo e i nomi di tutti gli altri elementi. Quindi imposto un punteggio minimo "di somiglianza" come valore soglia per fare dei confronti;

- Guardo l'output e valuto le eventuali modifiche da fare.

In [14]:
# Lista da testare con tutti i nomi unici dei paesi
test = df.Area.unique().tolist()

for country in test:
    
    # Dopo qualche test ho valutato che conviene fare un confronto tra i primi 3 elementi (limit=3)...
    matches = fuzzywuzzy.process.extract(country, test, limit=3, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    treshold = matches[1][1]
    
    # ...ed imporre il valore soglia >= 80!
    if treshold >= 80:
        print(matches)

[('Australia', 100), ('Austria', 88), ('Mauritania', 63)]
[('Austria', 100), ('Australia', 88), ('Costa Rica', 59)]
[("Democratic People's Republic of Korea", 100), ("Lao People's Democratic Republic", 87), ('Republic of Korea', 63)]
[('Gambia', 100), ('Zambia', 83), ('Namibia', 77)]
[('Iceland', 100), ('Ireland', 86), ('Finland', 71)]
[('Ireland', 100), ('Iceland', 86), ('Finland', 71)]
[("Lao People's Democratic Republic", 100), ("Democratic People's Republic of Korea", 90), ('Dominican Republic', 56)]
[('Malawi', 100), ('Mali', 80), ('Malta', 73)]
[('Mali', 100), ('Malawi', 80), ('Malaysia', 67)]
[('Niger', 100), ('Nigeria', 83), ('Algeria', 50)]
[('Nigeria', 100), ('Niger', 83), ('Algeria', 71)]
[('Zambia', 100), ('Gambia', 83), ('Namibia', 77)]


Come si può vedere, non ci sono pesanti incongruenze da sistemare tra i nomi dei paesi. C'è giusto qualche nome da ritoccare.

In [15]:
# Creo una lista con i paesi che presentano parentesi nell' etichetta del nome 
bracket = [country for country in df.Area.unique().tolist() if '(' in country.lower()]

# Creo una lista in cui inserire il loro nome seprato da tutto il resto
split_bracket = [country.split()[0] for country in bracket]

# Print informativo
print(f'Lista dei paesi con parentesi:\n{bracket}\n\n\
Lista dei paesi dopo aver rimosso le parentesi:\n{split_bracket}')

# Sostituisco i nuovi nomi con un metodo simile a quello di prima
df.Area.replace({bracket[n]:split_bracket[n] for n in range(len(bracket))}, inplace=True)

Lista dei paesi con parentesi:
['Bolivia (Plurinational State of)', 'Iran (Islamic Republic of)', 'Venezuela (Bolivarian Republic of)']

Lista dei paesi dopo aver rimosso le parentesi:
['Bolivia', 'Iran', 'Venezuela']


In [16]:
# Cerco i Paesi che hanno 'of' nel nome
of_names = [country for country in df.Area.unique().tolist() if 'of' in country.lower()]

# Formatto i nomi assegnando nomenclature maggiormente note
of_formatted = ['North Korea', 'South Korea', 'Tanzania', 'USA']

# Print informativo
print(f"Lista dei paesi con 'of' nel nome:\n{of_names}\n\n\
Lista dei paesi con le nuove nomenclature:\n{of_formatted}")

# Sostituisco i nuovi nomi con un metodo simile a quello di prima
df.Area.replace({of_names[n]:of_formatted[n] for n in range(len(of_names))}, inplace=True)

Lista dei paesi con 'of' nel nome:
["Democratic People's Republic of Korea", 'Republic of Korea', 'United Republic of Tanzania', 'United States of America']

Lista dei paesi con le nuove nomenclature:
['North Korea', 'South Korea', 'Tanzania', 'USA']


In [17]:
# Aggiusto questo nome "C�te d'Ivoire" in "Ivory Coast". È un problema di VS Code? 
df.Area.replace({"C�te d'Ivoire":'Ivory Coast'}, inplace=True)

# Infine, riordino i nomi dei paesi nel dataframe
df.sort_values('Area', inplace=True)

##### Sistemo la colonna 'Item'

Diamo un'occhiata alle nomenclature dei cibi e dei mangimi presenti nel dataset. Vediamo se ci sono voci a comune tra cibi e mangimi, voci esclusive dei cibi e/o voci esclusive dei mangimi. 

In [18]:
# Creo un dataframe ordinato secondo le nomenclature degli alimenti/mangimi del dataframe PRODUZIONE 
sorted_prod = df.sort_values('Item')
sorted_all_food_feed = sorted_prod.Item.unique().tolist()
print(f"Totale dei nomi unici di CIBI e MANGIMI: {len(sorted_all_food_feed)}\n")

# Da questo ne ricavo un altro dataframe contenente SOLO i cibi e ne ricavo la lista ORDINATA
sorted_prod_FOOD = sorted_prod.loc[sorted_prod['Element'] == 'Food']
sorted_FOOD_list = sorted_prod_FOOD.Item.unique().tolist()

# Ora ne ottengo un altro contenente SOLO i mangimi, ricavandone la lista ORDINATA
sorted_prod_FEED = sorted_prod.loc[sorted_prod['Element'] == 'Feed']
sorted_FEED_list = sorted_prod_FEED.Item.unique().tolist()

# Lista di elementi a comune di food e feed
feed_food_common = [n for n in sorted_FOOD_list if n in sorted_FEED_list]
print('-'*130)
print(f"Elementi a comune tra CIBI e MANGIMI: {len(feed_food_common)}\n")
print(feed_food_common)

# Lista di elementi unici di food
unique_FOOD_list = [n for n in sorted_FOOD_list if n not in sorted_FEED_list]
print('-'*130)
print(f"Elementi esclusivi della lista dei CIBI non presenti in quella dei MANGIMI: {len(unique_FOOD_list)}\n")
print(unique_FOOD_list)

# Lista di elementi unici di feed
unique_FEED_list = [n for n in sorted_FEED_list if n not in sorted_FOOD_list]

if len(unique_FEED_list) > 0:

    print('-'*130)
    print(f'\nElementi esclusivi della lista dei MANGIMI non presenti in quella dei CIBI: {len(unique_FEED_list)}\n')
    print(unique_FEED_list)

else:
    print('-'*130)
    print(f'\nLa lista degli elementi esclusivi dei MANGIMI è vuota, quindi tutte le nomenclature dei MANGIMI sono già \
incluse tra quelle dei CIBI.')

Totale dei nomi unici di CIBI e MANGIMI: 115

----------------------------------------------------------------------------------------------------------------------------------
Elementi a comune tra CIBI e MANGIMI: 82

['Animal fats', 'Apples and products', 'Aquatic Plants', 'Aquatic Products, Other', 'Bananas', 'Barley and products', 'Beans', 'Butter, Ghee', 'Cassava and products', 'Cephalopods', 'Cereals - Excluding Beer', 'Cereals, Other', 'Coconuts - Incl Copra', 'Cottonseed', 'Crustaceans', 'Dates', 'Demersal Fish', 'Eggs', 'Fats, Animals, Raw', 'Fish, Body Oil', 'Fish, Liver Oil', 'Fish, Seafood', 'Freshwater Fish', 'Fruits - Excluding Wine', 'Fruits, Other', 'Grapefruit and products', 'Grapes and products (excl wine)', 'Groundnuts (Shelled Eq)', 'Honey', 'Maize and products', 'Marine Fish, Other', 'Meat', 'Meat, Other', 'Milk - Excluding Butter', 'Millet and products', 'Molluscs, Other', 'Mutton & Goat Meat', 'Oats', 'Offals', 'Offals, Edible', 'Oilcrops', 'Oilcrops Oil, Other',

Il punto critico è che dovrei avere *congruenza tra i nomi degli alimenti* dei due datasets!

Per non complicare eccessivamente la pulizia dei dati, mi preoccuperò solo di avere una coerenza tra *i primi 20* nomi più rilevanti del dataset delle emissioni ed i loro corrispettivi nel dataset della produzione.

Anche in questo caso sfrutterò *fuzzywuzzy* per facilitarmi il lavoro, mentre il resto lo ricaverò osservando gli output che ricavo.

In [25]:
# Diamo un'occhiata al dataframe delle emissioni
df_emiss.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),Eutrophying emissions per kilogram (gPO₄eq per kilogram),Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein),Freshwater withdrawals per 1000kcal (liters per 1000kcal),Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,11.23,8.638462,183.911552,371.076923,482.4,0.945482,1.907692,2.897446,7.6,5.846154,18786.2,14450.92308,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,35.07,49.394366,609.983722,3166.760563,2248.4,1.207271,6.267606,0.759631,2.8,3.943662,49576.3,69825.77465,13449.891480


In [20]:
# Devo prima ordinare il dataframe delle emissioni in ordire decrescente secondo la colonna 'Total_emissions'
emiss_sorted = df_emiss.sort_values('Total_emissions', ascending=False).head(20)

# Poi, per comodità, ordino alfabeticamente i 20 nomi...
emiss_sorted = emiss_sorted.sort_values('Food product')

# ...e ne estraggo la lista
top20_emission_sorted = emiss_sorted['Food product'].unique().tolist()
print(f"Nomi dei 20 alimenti più rilevanti in termini di emissioni totali:\n\n{top20_emission_sorted}")

Nomi dei 20 alimenti più rilevanti in termini di emissioni totali:

['Beef (beef herd)', 'Beef (dairy herd)', 'Cane Sugar', 'Cheese', 'Coffee', 'Dark Chocolate', 'Eggs', 'Fish (farmed)', 'Lamb & Mutton', 'Milk', 'Olive Oil', 'Palm Oil', 'Pig Meat', 'Poultry Meat', 'Rapeseed Oil', 'Rice', 'Shrimps (farmed)', 'Soybean Oil', 'Sunflower Oil', 'Tofu']


In [21]:
# traduttore di top20_emission: utile per contestualizzare meglio alcuni elementi
translate_ENG_ITA = {

    "Beef (beef herd)":"Carni bovine (mandrie di bovini)",
    "Beef (dairy herd)":"Carni bovine (mandrie da latte)",
    "Cane Sugar":"zucchero di canna",
    "Cheese":"Formaggio",
    "Coffee":"Caffè",
    "Dark Chocolate":"cioccolato fondente",
    "Eggs":"Uova",
    "Fish (farmed)":"Pesce (d'allevamento)",
    "Lamb & Mutton":"Agnello & Montone",
    "Milk":"Latte",
    "Olive Oil":"Olio d'oliva",
    "Palm Oil":"olio di palma",
    "Pig Meat":"carne suina",
    "Poultry Meat":"Carni di pollame",
    "Rapeseed Oil":"Olio di colza",
    "Rice":"Riso",
    "Shrimps (farmed)":"Gamberetti (d'allevamento)",
    "Soybean Oil":"olio di soia",
    "Sunflower Oil":"Olio di girasole",
    "Tofu":"Tofu"
    
    }

In [37]:
#TODO ricava valori da questo loop e costruisci un dizionario, modifica da qui

# Counter utile per aiutare la lettura dell'output
counter = 0

# Dizionario per raccogliere le voci
replace_20dict = {}

for item in top20_emission_sorted:
    
    matches_ls = fuzzywuzzy.process.extract(item, sorted_all_food_feed, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Costruzione del dizionario, le chiavi (quindi i nomi da rimpiazzare in seguito) saranno quelle provienti 
    # dal dataframe della produzione di cibo
    key = matches_ls[0][0]

    # Costruzione del dizionario, i valori (ovvero i nomi degli alimenti del dataset delle emissioni) 
    # saranno i nuovi nomi degli alimenti
    replace_20dict[key] = item


    counter += 1
    print(f"{counter} voce: {item}")
    print(matches_ls)
    print('-'*60)
    print('\n')

print(f"Dizionario per rimpiazzare le voci:\n{replace_20dict}")

1 voce: Beef (beef herd)
[('Beer', 44), ('Cereals, Other', 44), ('Meat, Other', 42), ('Sugar beet', 42), ('Sesame seed', 40), ('Cereals - Excluding Beer', 39), ('Fruits, Other', 38), ('Barley and products', 36), ('Sunflower seed', 36), ('Groundnuts (Shelled Eq)', 34)]
------------------------------------------------------------


2 voce: Beef (dairy herd)
[('Sugar beet', 48), ('Cereals, Other', 43), ('Beer', 42), ('Oilcrops Oil, Other', 42), ('Meat, Other', 40), ('Groundnuts (Shelled Eq)', 39), ('Aquatic Animals, Others', 38), ('Cereals - Excluding Beer', 38), ('Sesame seed', 38), ('Fruits, Other', 37)]
------------------------------------------------------------


3 voce: Cane Sugar
[('Sugar cane', 100), ('Sugar beet', 70), ('Sugar Crops', 67), ('Sugar non-centrifugal', 58), ('Sugar (Raw Equivalent)', 53), ('Miscellaneous', 43), ('Sunflower seed', 42), ('Beans', 40), ('Dates', 40), ('Rape and Mustard Oil', 40)]
------------------------------------------------------------


4 voce: Che

In [ ]:
# {    
#     'Mutton & Goat Meat': 'Lamb & Mutton', 
#     'Sugar cane': 'Cane Sugar', 
#     'Palm Oil': 'Palm Oil', 
#     'Poultry Meat': 'Poultry Meat', 
#     'Olive Oil': 'Olive Oil', 
#     'Soyabean Oil': 'Soybean Oil', 
#     'Eggs': 'Eggs', 
#     'Sunflowerseed Oil': 'Sunflower Oil',
#     }

In [24]:
meat = df.Item.map(lambda text: 'meat' in text.lower())
beef = df.Item.map(lambda text: 'beef' in text.lower())

prod_meat_list = df.loc[meat].Item.unique()
prod_beef_list = df.loc[beef].Item.unique()

print(f"Elementi con la dicitura 'meat' nel dataframe della produzione:\n{prod_meat_list}\n")
print(f"Elementi con la dicitura 'beef' nel dataframe della produzione:\n{prod_beef_list}\n")

meat = emiss_sorted['Food product'].map(lambda text: 'meat' in text.lower())
beef = emiss_sorted['Food product'].map(lambda text: 'beef' in text.lower())

emiss_meat_list = emiss_sorted.loc[meat]['Food product'].unique()
emiss_beef_list = emiss_sorted.loc[beef]['Food product'].unique()

print(f"Elementi con la dicitura 'meat' nel dataframe delle emissioni:\n{emiss_meat_list}\n")
print(f"Elementi con la dicitura 'beef' nel dataframe delle emissioni:\n{emiss_beef_list}")

Elementi con la dicitura 'meat' nel dataframe della produzione:
['Meat, Other' 'Poultry Meat' 'Mutton & Goat Meat' 'Bovine Meat' 'Meat'
 'Pigmeat' 'Meat, Aquatic Mammals']

Elementi con la dicitura 'beef' nel dataframe della produzione:
[]

Elementi con la dicitura 'meat' nel dataframe delle emissioni:
['Pig Meat' 'Poultry Meat']

Elementi con la dicitura 'beef' nel dataframe delle emissioni:
['Beef (beef herd)' 'Beef (dairy herd)']


#### Termino la procedura di data cleaning per questo dataset sistemando l'etichetta di alcune colonne ed eliminando quelle che non sono utili alla mia analisi. 

In [ ]:
# # Rimuovo la 'Y' dalle labels degli anni con un dictionary comprehension
# df.rename(columns={n:n[1:] for n in df.columns if 'Y' in n}, inplace=True)

# # Rimuovo gli spazi presenti nelle labels
# df.rename(columns={n:n.replace(' ', '_') for n in df.columns if ' ' in n}, inplace= True)

# # Elimino le colonne non più necessarie per il proseguo dell'analisi
# df.drop(columns=['Area_Abbreviation', 'Area_Code', 'Element_Code', 'Unit'], inplace=True)

### TODO here
Dataset delle emissioni di gas serra e dei consumi di acqua

TRASFORMA IN CARATTERE 3###